In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import ( Input, Embedding, Flatten, Dot, Add, Concatenate, 
        Dense, Dropout, Activation, Multiply )
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [15]:
ratings = pd.read_csv('rratings.csv', sep=',',encoding='latin-1')
ratings.head()

,userId,routeId,rating
0,cjsysx7la00en0836zceujv79,cjsyxzoer01j908824mfq6lg6,1.2
1,cjsysx7la00en0836zceujv79,cjsyxzo7b01ba0882kjs0ta26,4.0
2,cjsysx7la00en0836zceujv79,cjsyxzo7m01c10882s8gmo2rt,2.7
3,cjsysx7la00en0836zceujv79,cjsyxzodf01h40882qzwjsdqx,1.2
4,cjsysx7la00en0836zceujv79,cjsyxzod601ga088290hvk8fb,3.5


In [16]:
u_unique = ratings.userId.unique()
user2Id = { userId : index for index, userId in enumerate(u_unique)}
ratings.userId = ratings.userId.apply(lambda x: user2Id[x])

r_unique = ratings.routeId.unique()
route2Id = { routeId : index for index, routeId in enumerate(r_unique)}
ratings.routeId = ratings.routeId.apply(lambda x: route2Id[x])

In [17]:
n_users = ratings.userId.nunique()
n_routes = ratings.routeId.nunique()
n_users, n_routes

(103, 376)

In [18]:
n_factors = 50

In [19]:
np.random.seed = 42

In [20]:
msk = np.random.rand(len(ratings)) < 0.8

In [21]:
trn = ratings[msk]
val = ratings[~msk]
val.shape

(221, 3)

In [22]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

# Neural Network Model

In [23]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
route_in, r = embedding_input('route_in', n_routes, n_factors, 1e-5)

Instructions for updating:
Colocations handled automatically by placer.


In [39]:
x = Concatenate()([u, r])
x = Flatten()(x)
x = Dense(units=100, activation='relu')(x)
x = Dropout(rate=0.75)(x)
x = Dense(1)(x)
model = tf.keras.Model(inputs=[user_in, route_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [40]:
model.fit([trn.userId, trn.routeId], trn.rating, batch_size=64, epochs=6,
          validation_data=([val.userId, val.routeId], val.rating))

Train on 797 samples, validate on 221 samples
Epoch 1/6
797/797 [==============================] - 1s 791us/sample - loss: 5.3178 - val_loss: 5.0924
Epoch 2/6
797/797 [==============================] - 0s 73us/sample - loss: 3.5189 - val_loss: 3.6172
Epoch 3/6
797/797 [==============================] - 0s 92us/sample - loss: 2.0571 - val_loss: 2.4638
Epoch 4/6
797/797 [==============================] - 0s 82us/sample - loss: 1.2592 - val_loss: 2.0235
Epoch 5/6
797/797 [==============================] - 0s 74us/sample - loss: 0.9900 - val_loss: 2.0750
Epoch 6/6
797/797 [==============================] - 0s 85us/sample - loss: 0.9434 - val_loss: 2.1716
